In [7]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline

In [21]:
class NeuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.lr = learningrate
        
        # Factor matrix: wih (weight (input-hidden)), who (weight (hidden-output))
        # Weight coefficients of connections between i node and j node of next layer = w_i_j
        # w11 w 21
        # w12 w 22 etc
        
        # Center of normal distribution = 0.0
        # Standart deviation: calculated based on number of nodes in the next layer -->
        # -- > (pow(self.hnodes, - 0.5)) - raises the number of nodes to a power of -0.5
        # Last parameter determines numpy array configuration

        self.wih = numpy.random.normal(0.0, pow(self.hnodes, - 0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, - 0.5), (self.onodes, self.hnodes))
        
        # Sigmoid (expit) used as activation function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass
    
    # neuralnet training
    def train(self, inputs_list, targets_list):
        
        # Training implies two phases: calculation of output signal (query() func) and errors backpropagation -->
        # --> which informs us about what corrections need to be applied to weight coefficents
        
        # Transform input values into two-dimensional array (ndmin = 2 -- two-dimensional)
        # .T - transpose
        inputs = numpy.array(inputs_list, ndmin = 2).T
        targets = numpy.array(targets_list, ndmin = 2).T
        
        # Calculate input signals for the hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
          
        # Calculate output signals for the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # Calculate input signals for the output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        #Calculate output signals for the output layer
        final_outputs = self.activation_function(final_inputs)
        
        # Error = target value - actual value
        output_errors = targets - final_outputs
        
        # Ehidden = WEIGHTS.Thidden_output * Eoutput
        # Errors in the hidden layer are output_errors, distributed proportionally to weight coefficients -->
        # --> of connections and recombinated to hidden layers
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # Renew weight coefficients of connections between input and hidden layers
        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # Renew weight coefficients of connections between hidden and output layers
        self.wih += self.lr * numpy.dot(
            (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass
    
    # neuralnet query
    # Takes input data as an argument and returns its output data
    def query(self, inputs_list):
        
        # Transform input values into two-dimensional array
        inputs = numpy.array(inputs_list, ndmin = 2).T
        
        # Xhidden = Winput_hiiden * I(nput)
        # Calculate input signals for the hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        
        # To get output signals of the hidden layer we nned to implement sigmoid function on those signals
        # O(utput)hidden = sigmoid(Xhidden)
        
        # Calculate output signals for the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # Calculate input signals for the output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        
        #Calculate output signals for the output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [22]:
# 784 = 28x28 (pixels in handwritten number)
input_nodes = 784

# Number of hidden nodes must be smaller than the number of input nodes as -->
# --> net needs to train to find key differences through information generalization
# Random! Depends on the number of output nodes
hidden_nodes = 100

# Output will ne numbers 0-9
output_nodes = 10

learning_rate = 0.3

net = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [23]:
training_data_file = open("mnist_train_100.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

In [26]:
# Neuralnet training

# Iterate over all values in training data set
for record in training_data_list:
    all_values = record.split(",")
    
    # Scale and shift input values
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    # Create target values
    targets = numpy.zeros(output_nodes) + 0.01
    
    # all_values[0] is a target value
    targets[int(all_values[0])] = 0.99
    net.train(inputs,targets)
    
    pass

In [129]:
# Neuralnet testing

test_data_file = open("mnist_test_10.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

In [130]:
# Journal of neuralnet work appraisal
scorecard = []

# Iterate over all values in training data set
for record in test_data_list:
    all_values = record.split(',')

    # Correct value is the first value
    correct_label = int(all_values[0])
    print("True marker:", correct_label)
    
    # Scale and shift input values
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    # Neuralnet query
    outputs = net.query(inputs)
    
    # Index of the biggest value is the marker value
    label = numpy.argmax(outputs)
    print("Neuralnet answer:", label)
    
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    
    pass

True marker: 7
Neuralnet answer: 7
True marker: 2
Neuralnet answer: 2
True marker: 1
Neuralnet answer: 1
True marker: 0
Neuralnet answer: 0
True marker: 4
Neuralnet answer: 4
True marker: 1
Neuralnet answer: 1
True marker: 4
Neuralnet answer: 4
True marker: 9
Neuralnet answer: 9
True marker: 5
Neuralnet answer: 6
True marker: 9
Neuralnet answer: 9


In [131]:
# Calculate neuralnet efficiency
scorecard_array = numpy.asarray(scorecard)
print ("Efficiency = ", scorecard_array.sum() / scorecard_array.size)

Efficiency =  0.9
